In [1]:
proj_list = [
    'boringssl_total', 'c-ares_total',
    'freetype2_total', 'guetzli_total',
    'harfbuzz_total', 'libpng_total',
    'libssh_total', 'libxml2_total',
    'pcre_total', 'proj4_total',
    're2_total', 'sqlite3_total',
    'total', 'vorbis_total',
    'woff2_total', 'wpantund_total'
]

In [2]:
from sklearn.model_selection import train_test_split
import torch

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import data_loader as dl
import initializer as init
import trainer
import tester
import predictor
import model_util as mu
import pretrained_model as pm

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
prefix_np, postfix_np, label_np = data.getSingleProjectData(proj_list, proj_list[0])

Getting data for "boringssl_total" from "c-ares_total"
Getting data for "boringssl_total" from "freetype2_total"
Getting data for "boringssl_total" from "guetzli_total"
Getting data for "boringssl_total" from "harfbuzz_total"
Getting data for "boringssl_total" from "libpng_total"
Getting data for "boringssl_total" from "libssh_total"
Getting data for "boringssl_total" from "libxml2_total"
Getting data for "boringssl_total" from "pcre_total"
Getting data for "boringssl_total" from "proj4_total"
Getting data for "boringssl_total" from "re2_total"
Getting data for "boringssl_total" from "sqlite3_total"
Getting data for "boringssl_total" from "vorbis_total"
Getting data for "boringssl_total" from "woff2_total"
Getting data for "boringssl_total" from "wpantund_total"


In [4]:
train_prefix, test_prefix, train_postfix, test_postfix, train_label, test_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.1, random_state = 43
)

train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    train_prefix, train_postfix, train_label, test_size = 0.1, random_state = 43
)

In [5]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [6]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/dev/tests')

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [18]:
# ====================
# set parameters here
# ====================

title = 'dev-test3'
epochs = 30

embed_dim = 50
max_len, source_code_tokens, token_choices = data.getInfo()
pretrained_token2vec = pm.load_pretrained_model(source_code_tokens, embed_dim)
pretrained_token2vec = torch.tensor(pretrained_token2vec)


input_size = max_len
hidden_size = 30
num_classes = max(token_choices) + 1
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.0

learning_rate = 0.001
weight_decay = 1e-4

model_name = "RNN"
optim_name = "Adam"
loss_fn_name = "CEL"

pretrained_model = pretrained_token2vec
freeze_embedding = False,

In [19]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=input_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
C_rnn(
  (emb): Embedding(213, 50)
  (lstm1): RNN(
    (rnn): LSTM(50, 30, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(50, 30, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=120, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=213, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [20]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.947500   | 73.766231  | 0.552750 | 83.25  | 94.06 
   2    |   0.478113   | 85.454797  | 0.434125 | 86.53  | 93.96 
   3    |   0.404514   | 87.348740  | 0.383230 | 87.94  | 98.97 
   4    |   0.366785   | 88.337888  | 0.354688 | 88.66  | 94.21 
   5    |   0.343877   | 88.973983  | 0.334278 | 89.13  | 92.81 
   6    |   0.328697   | 89.386095  | 0.319913 | 89.65  | 93.78 
   7    |   0.317627   | 89.713130  | 0.312544 | 89.85  | 94.13 
   8    |   0.308885   | 89.963421  | 0.313861 | 89.75  | 93.63 
   9    |   0.302319   | 90.153488  | 0.303441 | 90.15  | 93.81 
  10    |   0.296600   | 90.326744  | 0.294913 | 90.38  | 93.55 
  11    |   0.291386   | 90.482994  | 0.297056 | 90.23  | 93.60 
  12    |   0.287358   | 90.600291  | 0.283157 | 90.73  | 94.30 
  13    |   0.283830   | 90.695736  | 0.286588 | 90.7

In [21]:
mu.saveModel(title, model)

In [4]:
model = mu.getModel('dev-test3')
print(model)

C_rnn(
  (emb): Embedding(213, 50)
  (lstm1): RNN(
    (rnn): LSTM(50, 30, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(50, 30, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=120, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=213, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [29]:
tester.test(test_dataloader=test_dataloader,
            device=device,
            model=model,
            title=title)

test loss:  0.2571676188683885
test acc:  91.5996062992126


/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


saved precision and recall results to file!


(tensor([ 48.,  50.,  62.,  ..., 134.,   2.,  91.], device='cuda:0'),
 tensor([ 48.,  50.,  62.,  ..., 134.,   2.,  91.], device='cuda:0'))

In [30]:
mu.graphModel(train_dataloader, model, writer)

: 

: 

In [25]:
prefix =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
postfix = [2, 91, 2, 56, 2, 106, 47, 2, 134, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 91, 2, 119, 91, 2, 56, 2, 106, 47, 2, 119, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 91, 2, 119, 91, 2, 56, 2, 106, 47, 2, 119, 128, 50, 88, 33, 124, 49, 48, 134, 47, 2, 119]
label_type = 128

In [5]:
prefix =[0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 119, 2, 47, 134, 48, 49, 124, 33, 88, 50, 128, 119, 2, 47, 106, 2, 56, 2, 91, 119, 2, 91, 2, 56, 2, 48, 49, 2, 47, 48, 88, 50, 128, 119, 2, 47, 106, 2, 56, 2, 91, 119, 2, 91, 2, 56, 2, 48, 49, 2, 47, 48, 88, 50]
postfix = [48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134]
label_type = 128

In [6]:
predictor.predict(prefix, postfix, model)

#1) 128: 85.76244354248047%
#2) 50: 8.61021614074707%
#3) 118: 1.3333876132965088%
#4) 106: 1.088342547416687%
#5) 113: 0.9660220742225647%
#6) 88: 0.7549131512641907%
#7) 2: 0.5122286081314087%
#8) 131: 0.17010332643985748%
#9) 134: 0.09199777245521545%
#10) 133: 0.07557959109544754%
